In [1]:
import collections
import copy
import os

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from absl import app, flags
from skimage import io
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

task_type = 'training'
experiment_name = 'exp'
label_type = 'domain'
learning_rate = 1e-4
weight_decay = 0
batch_size  = 256
epochs = 5
LABEL_SIZE = {'domain': 4, 'category': 7}

class PACSDataset(Dataset):

  def __init__(self,
               root_dir,
               label_type='domain',
               is_training=False,
               transform=None):
    self.root_dir = os.path.join(root_dir, 'train' if is_training else 'val')
    self.label_type = label_type
    self.is_training = is_training
    if transform:
      self.transform = transform
    else:
      self.transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.7659, 0.7463, 0.7173],
                               std=[0.3089, 0.3181, 0.3470]),
      ])

    self.dataset, self.label_list = self.initialize_dataset()
    self.label_to_id = {x: i for i, x in enumerate(self.label_list)}
    self.id_to_label = {i: x for i, x in enumerate(self.label_list)}

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    image, label = self.dataset[idx]
    label_id = self.label_to_id[label]
    image = self.transform(image)
    return image, label_id

  def initialize_dataset(self):
    assert os.path.isdir(self.root_dir), \
        '`root_dir` is not found at %s' % self.root_dir

    dataset = []
    domain_set = set()
    category_set = set()
    cnt = 0

    for root, dirs, files in os.walk(self.root_dir, topdown=True):
      if files:
        parts = root.split(os.sep)
        if len(parts) >= 3:
          domain = parts[-2]
          category = parts[-1]
          domain_set.add(domain)
          category_set.add(category)
          pbar = tqdm(files)
          for name in pbar:
            pbar.set_description('Processing Folder: domain=%s, category=%s' %
                                (domain, category))
            img_array = io.imread(os.path.join(root, name))
            dataset.append((img_array, domain, category))
    # use this below if you are running on google colab
    # for root, dirs, files in os.walk(self.root_dir, topdown=True):
    #   if files:
    #     _, domain, category = root.rsplit('/', maxsplit=2)
    #     domain_set.add(domain)
    #     category_set.add(category)
    #     pbar = tqdm(files)
    #     for name in pbar:
    #       pbar.set_description('Processing Folder: domain=%s, category=%s' %
    #                            (domain, category))
    #       img_array = io.imread(os.path.join(root, name))
    #       dataset.append((img_array, domain, category))
    images, domains, categories = zip(*dataset)

    if self.label_type == 'domain':
      labels = sorted(domain_set)
      dataset = list(zip(images, domains))
    elif self.label_type == 'category':
      labels = sorted(category_set)
      dataset = list(zip(images, categories))
    else:
      raise ValueError(
          'Unknown `label_type`: Expecting `domain` or `category`.')

    return dataset, labels


class AlexNet(nn.Module):
    def __init__(self, configs):
        super(AlexNet, self).__init__()
        dropout = configs['dropout']
        num_classes = configs['num_classes']

        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout),
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def model_training():

  best_model = None
  best_acc = 0.0

  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

  expt_name = 'experiments/{}/{}_lr_{}.wd_{}'.format(experiment_name, label_type, learning_rate, weight_decay)

  os.makedirs(expt_name, exist_ok=True)
  writer = SummaryWriter(log_dir=expt_name)

  configs = {'num_classes': LABEL_SIZE[label_type], 'dropout': 0.5}

  model = AlexNet(configs).to(device)

  print('Model Architecture:\n%s' % model)

  criterion = nn.CrossEntropyLoss(reduction='mean')
  optimizer = torch.optim.Adam(model.parameters(),
                               lr=learning_rate,
                               weight_decay=weight_decay)

  try:
    for epoch in range(epochs):
      print(epoch)
      for phase in ('train', 'eval'):
        if phase == 'train':
          model.train()
          dataset = train_dataset
          data_loader = train_loader
        else:
          model.eval()
          dataset = val_dataset
          data_loader = val_loader

        running_loss = 0.0
        running_corrects = 0

        for step, (images, labels) in enumerate(data_loader):
          print(step)
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()

          with torch.set_grad_enabled(phase == 'train'):
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            if phase == 'train':
              loss.backward()
              optimizer.step()

              writer.add_scalar('Loss/{}'.format(phase), loss.item(),
                                epoch * len(data_loader) + step)

          running_loss += loss.item() * images.size(0)
          running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataset)
        epoch_acc = running_corrects.double() / len(dataset)
        writer.add_scalar('Epoch_Loss/{}'.format(phase), epoch_loss, epoch)
        writer.add_scalar('Epoch_Accuracy/{}'.format(phase), epoch_acc, epoch)
        print('[Epoch %d] %s accuracy: %.4f, loss: %.4f' %
              (epoch + 1, phase, epoch_acc, epoch_loss))

        if phase == 'eval':
          if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, os.path.join(expt_name, 'best_model.pt'))

  except KeyboardInterrupt:
    pass

  return


In [2]:
# Download the data

!wget https://people.cs.pitt.edu/~mzhang/cs1699/pacs_dataset.zip
!unzip pacs_dataset.zip


'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
  # Make sure to run this before training the model

  train_dataset = PACSDataset(root_dir='pacs_dataset',
                              label_type=label_type,
                              is_training=True)
  train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=0)

  val_dataset = PACSDataset(root_dir='pacs_dataset',
                            label_type=label_type,
                            is_training=False)
  val_loader = DataLoader(val_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=0)

Processing Folder: domain=art_painting, category=dog: 100%|██████████| 348/348 [00:00<00:00, 358.89it/s]
Processing Folder: domain=art_painting, category=elephant: 100%|██████████| 227/227 [00:00<00:00, 364.33it/s]
Processing Folder: domain=art_painting, category=giraffe: 100%|██████████| 254/254 [00:00<00:00, 365.87it/s]
Processing Folder: domain=art_painting, category=guitar: 100%|██████████| 169/169 [00:00<00:00, 379.92it/s]
Processing Folder: domain=art_painting, category=horse: 100%|██████████| 179/179 [00:00<00:00, 384.82it/s]
Processing Folder: domain=art_painting, category=house: 100%|██████████| 262/262 [00:00<00:00, 366.07it/s]
Processing Folder: domain=art_painting, category=person: 100%|██████████| 404/404 [00:01<00:00, 344.71it/s]
Processing Folder: domain=cartoon, category=dog: 100%|██████████| 343/343 [00:00<00:00, 402.09it/s]
Processing Folder: domain=cartoon, category=elephant: 100%|██████████| 411/411 [00:01<00:00, 391.47it/s]
Processing Folder: domain=cartoon, catego

In [5]:
# Train the model and output accuracy

model_training()

Model Architecture:
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=9216, out_features=4096, bias=True)
    (3

[Epoch 5] train accuracy: 0.8517, loss: 0.3467
[Epoch 5] eval accuracy: 0.8527, loss: 0.3052

With a smaller learning rate adjustment to the 1e-04, and a larger batch size of 256, the evaluation accuracy on the validation dataset comes to  0.84.